# TikTok Order Data Management for Piwpiw Bakery

## 1. Project Overview

**Objective:**  
Develop a comprehensive system for managing TikTok orders at Piwpiw Bakery, addressing challenges related to data storage, order tracking, and handling returns.

## 2. Requirements Analysis

**Business Requirements:**  
- Store TikTok order data and labels.
- Track users with a history of parcel returns.
- Automate data entry, updating, and reporting.

**Technical Requirements:**  
- Google Sheets and Google Drive APIs for data storage.
- Python for scripting and automation.
- ETL process for data transformation.

## 3. Design Specifications

**a. Data Collection and Storage**

- **Data Sources:**  
  TikTok orders, PDF labels.

- **Data Extraction:**  
  Use PyMuPDF for extracting labels from PDFs.
  
- **Data Storage:**  
  Store extracted data in Google Sheets.

- **APIs:**  
  - Google Sheets API for data entry and retrieval.
  - Google Drive API for file management.

**b. ETL Process**

- **Extract:**  
  Extract data from PDF labels and existing Google Sheets.

- **Transform:**  
  Process and clean data:
  - Check for and handle duplicate order IDs.
  - Merge new data with existing records.

- **Load:**  
  Update Google Sheets with processed data.

**c. Workflow Automation**

- **Data Entry and Updating:**  
  Develop Python scripts to automate:
  - Data extraction from PDFs.
  - Data entry and updates in Google Sheets.
  - Consistency checks across datasets.

**d. Order Monitoring and Reporting**

- **Order Tracking:**  
  Script to:
  - Monitor new orders.
  - Identify users with a history of parcel returns.

- **Automated Reporting:**  
  Script to:
  - Generate reports on problematic orders.
  - Send automated notifications to Piwpiw Bakery for follow-up.

## 4. Implementation Plan

**a. Development:**

- **Phase 1: Setup**
  - Configure APIs (Google Sheets, Google Drive).
  - Install necessary Python libraries (PyMuPDF, gspread).

- **Phase 2: Data Extraction and Storage**
  - Develop and test scripts for extracting data from PDFs.
  - Implement Google Sheets integration.

- **Phase 3: ETL Process**
  - Create and test ETL scripts for data transformation.

- **Phase 4: Automation**
  - Develop scripts for data entry, updating, and consistency maintenance.

- **Phase 5: Order Monitoring and Reporting**
  - Develop and test scripts for order tracking and reporting.

**b. Testing:**

- **Unit Testing:**  
  Test individual scripts and functions.

- **Integration Testing:**  
  Ensure all components work together seamlessly.

- **User Acceptance Testing:**  
  Validate with Piwpiw Bakery to ensure requirements are met.

**c. Deployment:**

- **Deploy Scripts:**  
  Deploy and schedule Python scripts on a server or cloud platform.

- **Training:**  
  Train bakery staff on using the new system.

## 5. Maintenance and Support

- **Monitoring:**  
  Regularly monitor system performance and data accuracy.

- **Updates:**  
  Implement updates and improvements based on feedback.

- **Support:**  
  Provide ongoing support and troubleshooting as needed.


# Install Package

In [1]:
pip install gspread oauth2client pandas numpy


  Using cached gspread-6.1.2-py3-none-any.whl.metadata (11 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
     ---------------------------------------- 0.0/49.3 kB ? eta -:--:--
     ---------------------------------------- 49.3/49.3 kB 2.6 MB/s eta 0:00:00
Using cached gspread-6.1.2-py3-none-any.whl (57 kB)
Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
   ---------------------------------------- 0.0/143.9 kB ? eta -:--:--
   ----------------- ---------------------- 61.4/143.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 143.9/143.9 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 12.0 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
torchvision 0.15.2 requires torch==2.0.1, but you have torch 2.1.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Authentication and Connection

Authenticates using OAuth2 credentials and connects to Google Sheets using the gspread library.

In [ ]:

# Path to your service account key file
CLIENT_SECRET_FILE = os.getenv('CLIENT_SECRET_FILE')

API_NAME = 'drive'
API_VERSION = 'v3'
# Scopes for the APIs
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']


## Checking for Duplicate Order IDs and Updating Orders

ETL Process Overview: 

Extract: Data is pulled from various sources, including multiple Google Sheets and Google Drive.

Transform: Data is cleaned and filtered to identify non-duplicate records, ensuring data quality and consistency.

Load: The processed data is stored in target Google Sheets, and the process is logged for auditing and tracking purposes.
By integrating these phases, the script helps maintain a clean, up-to-date dataset, essential for further analysis and decision-making processes.

In [ ]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
from datetime import datetime
from googleapiclient.discovery import build
import os


# Create a credentials object
creds = Credentials.from_service_account_file(CLIENT_SECRET_FILE, scopes=SCOPES)

# Initialize gspread client
client = gspread.authorize(creds)

# Initialize Google Drive client
drive_service = build('drive', 'v3', credentials=creds)

def get_sheet_data(sheet_url, sheet_name, file_name):
    """Fetch data from a Google Sheet and add file name to the DataFrame."""
    try:
        sheet = client.open_by_url(sheet_url).worksheet(sheet_name)
        print(f"Sheet name: {sheet.title}")
        records = sheet.get_all_records()
        if not records:
            print(f"Sheet {sheet_name} is empty.")
            return pd.DataFrame()
        
        df = pd.DataFrame(records)
        df['File Name'] = file_name
        return df
    except gspread.exceptions.APIError as e:
        print(f"APIError fetching data from {sheet_name}: {e}")
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"WorksheetNotFound error fetching data from {sheet_name}: {e}")
    except gspread.exceptions.SpreadsheetNotFound as e:
        print(f"SpreadsheetNotFound error fetching data from {sheet_name}: {e}")
    except Exception as e:
        print(f"General error fetching data from {sheet_name}: {e}")
    return pd.DataFrame()

def write_to_sheet(sheet_url, sheet_name, data):
    """Write data to a Google Sheet."""
    try:
        sheet = client.open_by_url(sheet_url).worksheet(sheet_name)
        sheet.clear()
        if not data.empty:
            # Convert data to a list of lists and ensure numbers are formatted as strings
            formatted_data = data.applymap(lambda x: f"{x:.0f}" if isinstance(x, (int, float)) else x)
            # Update the sheet with column headers and data
            sheet.update([formatted_data.columns.values.tolist()] + formatted_data.values.tolist())
        else:
            print(f"No data to write to {sheet_name}.")
    except gspread.exceptions.APIError as e:
        print(f"APIError writing data to {sheet_name}: {e}")
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"WorksheetNotFound error writing data to {sheet_name}: {e}")
    except gspread.exceptions.SpreadsheetNotFound as e:
        print(f"SpreadsheetNotFound error writing data to {sheet_name}: {e}")
    except Exception as e:
        print(f"General error writing data to {sheet_name}: {e}")

def compare_order_ids_and_filter(master_df, update_df):
    """Compare Order IDs and return non-duplicate rows with Date and Timestamp, and count of orders checked."""
    # Clean 'Order ID' columns in both DataFrames
    master_df['Order ID'] = master_df['Order ID'].astype(str).str.strip()
    update_df['Order ID'] = update_df['Order ID'].astype(str).str.strip()
    
    # Remove header rows if present in the 'Order ID' column
    master_df = master_df[master_df['Order ID'] != 'Platform unique order ID.']
    update_df = update_df[update_df['Order ID'] != 'Platform unique order ID.']
    
    # Print unique Order IDs for debugging
    print(f"Unique Order IDs in Master Sheet: {master_df['Order ID'].unique()}")
    print(f"Unique Order IDs in Update Sheet: {update_df['Order ID'].unique()}")
    
    # Create a set of 'Order ID' from master_df for faster lookup
    master_ids = set(master_df['Order ID'])
    
    # Filter out rows in update_df where 'Order ID' is not in master_ids
    non_duplicates = update_df[~update_df['Order ID'].isin(master_ids)]
    
    # Print the filtered non-duplicate Order IDs for debugging
    print(f"Non-Duplicate Order IDs: {non_duplicates['Order ID'].unique()}")
    
    # Add 'Date' and 'Timestamp' columns
    non_duplicates['Date Check Update'] = datetime.now().strftime('%Y-%m-%d')
    non_duplicates['Timestamp Check Update'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Ensure unique 'Order ID' values in non_duplicates
    non_duplicates = non_duplicates.drop_duplicates(subset='Order ID')
    
    return non_duplicates, len(update_df)

def log_update(sheet_url, sheet_name, file_name, order_count, non_duplicate_count):
    """Log the file name, date, timestamp, and count of orders checked and non-duplicate orders to the update log sheet."""
    try:
        # Open the spreadsheet by URL
        spreadsheet = client.open_by_url(sheet_url)
        print(f"Opened spreadsheet: {spreadsheet}")
        
        # Try to open the worksheet by name
        try:
            sheet = spreadsheet.worksheet(sheet_name)
            print(f"Accessed worksheet: {sheet.title}")
        except gspread.exceptions.WorksheetNotFound:
            print(f"Worksheet '{sheet_name}' not found in the spreadsheet.")
            # Optionally, list all sheet names to verify
            sheet_names = [ws.title for ws in spreadsheet.worksheets()]
            print(f"Available sheets: {sheet_names}")
            return
        
        # Fetch existing data
        existing_data = sheet.get_all_records()
        log_df = pd.DataFrame(existing_data)
        
        # Create a new log entry
        new_entry = {
            'File Name': file_name,
            'Date': datetime.now().strftime('%Y-%m-%d'),
            'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Order Count': order_count,
            'Non-Duplicate Order Count': non_duplicate_count
        }
        
        # Append new entry to DataFrame using pd.concat
        new_entry_df = pd.DataFrame([new_entry])
        log_df = pd.concat([log_df, new_entry_df], ignore_index=True)
        
        # Fill NaN values with an empty string to avoid JSON issues
        log_df = log_df.fillna('')
        
        # Update the worksheet with new data
        print(f"Log DataFrame to be written:\n{log_df}")  # Debug statement
        sheet.update([log_df.columns.values.tolist()] + log_df.values.tolist())
        print(f"Logged update for {file_name} with {order_count} orders checked and {non_duplicate_count} non-duplicate orders.")
    
    except gspread.exceptions.APIError as e:
        print(f"APIError logging update for {file_name}: {e}")
    except gspread.exceptions.SpreadsheetNotFound as e:
        print(f"SpreadsheetNotFound error logging update for {file_name}: {e}")
    except Exception as e:
        print(f"General error logging update for {file_name}: {e}")



def get_files_in_folder(folder_id):
    """Get list of files in a Google Drive folder."""
    try:
        results = drive_service.files().list(
            q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'",
            fields="files(id, name)"
        ).execute()
        files = results.get('files', [])
        print(f"Files found: {files}")  # Print the files retrieved
        return files
    except Exception as e:
        print(f"Error getting files from folder: {e}")
        return []

def process_google_sheet(file_id, file_name):
    """Process a Google Sheet directly from Drive."""
    try:
        # Construct the URL for the sheet
        sheet_url = f"https://docs.google.com/spreadsheets/d/{file_id}/edit"
        # Fetch the sheet data (assuming the first sheet/tab)
        sheet = client.open_by_url(sheet_url).sheet1
        records = sheet.get_all_records()
        if not records:
            print(f"Sheet with ID {file_id} is empty.")
            return pd.DataFrame()
        
        df = pd.DataFrame(records)
        df['File Name'] = file_name  # Add the file name to the DataFrame
        return df
    except Exception as e:
        print(f"Error processing Google Sheet with ID {file_id}: {e}")
        return pd.DataFrame()

# URLs of the Google Sheets (Replace with actual URLs)
MASTER_SHEET_URL = os.getenv('MASTER_SHEET_URL')
OUTPUT_SHEET_URL = os.getenv('OUTPUT_SHEET_URL')
LOG_SHEET_URL = os.getenv('LOG_SHEET_URL')

# Sheet names (tabs) in the Google Sheets
MASTER_SHEET_NAME = 'Master Sheet'
OUTPUT_SHEET_NAME = 'Update Order For Check Blacklist'
LOG_SHEET_NAME = 'Update Log'

# Google Drive folder ID containing the update files (replace with your folder ID)
UPDATE_FOLDER_ID = os.getenv('UPDATE_FOLDER_ID')
# Fetch data from master sheet
master_df = get_sheet_data(MASTER_SHEET_URL, MASTER_SHEET_NAME, 'Master Sheet')

if master_df.empty:
    print(f"Failed to fetch data from {MASTER_SHEET_NAME}.")
else:
    # Get list of files in the update folder
    files = get_files_in_folder(UPDATE_FOLDER_ID)
    if not files:
        print(f"No files found in folder with ID {UPDATE_FOLDER_ID}.")
    else:
        for file in files:
            file_id = file['id']
            file_name = file['name']
            
            print(f"Processing file: {file_name} (ID: {file_id})")
            
            # Process the Google Sheet directly
            update_df = process_google_sheet(file_id, file_name)
            
            if update_df is not None and not update_df.empty:
                # Compare Order IDs and get non-duplicate rows
                non_duplicates_df, order_count = compare_order_ids_and_filter(master_df, update_df)
                
                non_duplicate_count = len(non_duplicates_df)
                
                if not non_duplicates_df.empty:
                    # Write non-duplicate rows to output sheet
                    write_to_sheet(OUTPUT_SHEET_URL, OUTPUT_SHEET_NAME, non_duplicates_df)
                    print(f"Non-duplicate rows from {file_name} have been written to {OUTPUT_SHEET_NAME}.")
                
                # Log the update
                log_update(LOG_SHEET_URL, LOG_SHEET_NAME, file_name, order_count, non_duplicate_count)
                print(f"Logged update for {file_name} with {order_count} orders checked and {non_duplicate_count} non-duplicate orders.")
            else:
                print(f"Failed to process file {file_name}.")


## Extracting and Processing Data from PDF Labels

This script implements an ETL (Extract, Transform, Load) process to extract data from PDF labels stored in Google Drive, transform the data to extract relevant fields, and load the results into a Google Sheets document.

Extract Phase:

Google Drive File Retrieval: The script connects to Google Drive using the API, lists all files in a specified folder (folder_id), and retrieves them, handling pagination as needed.
PDF Text Extraction: For each PDF, it downloads the content, extracts the full text using PyMuPDF (fitz), and stores the text in a single string for further processing.

Transform Phase:

Data Structuring: The extracted information is organized into a DataFrame with columns for Name, Address, Order ID, Phone, Product, Date, and Timestamp.
Filtering Duplicate Entries: The script removes duplicate Order IDs by comparing them with existing IDs in the "Label Extract MasterSheet" on Google Sheets.

Load Phase:

Data Appending: Non-duplicate data is appended to the "Label Extract MasterSheet" in Google Sheets using the gspread library.
Logging and Monitoring: Status messages are printed throughout the process, tracking progress and facilitating debugging.

In [ ]:
import fitz  # PyMuPDF
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import gspread
from datetime import datetime

# Function to extract text from all pages of a PDF from a stream
def extract_text_from_pdf_stream(pdf_stream):
    document = fitz.open("pdf", pdf_stream)
    full_text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        page_text = page.get_text()
        full_text += page_text
    return full_text

# Function to split address into components
def split_address(address):
    parts = address.split(',')
    address_parts = [part.strip() for part in parts]
    # Ensure we have at least 7 components, fill with empty strings if less
    address_parts += [''] * (7 - len(address_parts))
    return address_parts[:7]

# Function to remove spaces from address components
def remove_spaces(address_part):
    return address_part.replace(" ", "")

# Function to extract name, address, order ID, phone, and products from DataFrame
def extract_name_address_order_phone_and_products(df):
    name = None
    address_lines = []
    order_id = None
    phone = None
    products = []
    extracting_products = False
    extracted_data = []

    for index, row in df.iterrows():
        lines = row['pdf_extract'].split('\n')
        for line in lines:
            if "ถึง" in line:
                try:
                    name = lines[lines.index(line) + 1].strip()
                    for address_line in lines[lines.index(line) + 2:]:
                        if "EZ" in address_line:
                            break
                        address_lines.append(address_line.strip())
                except IndexError:
                    pass
            elif "Order ID:" in line:
                try:
                    order_id = line.split("Order ID:")[1].strip()
                except IndexError:
                    pass
            elif "(+66)" in line:
                try:
                    phone = line.strip()
                except IndexError:
                    pass
            elif "Product Name" in line:
                extracting_products = True
                continue
            elif "Qty Total" in line:
                extracting_products = False
                continue
            elif extracting_products:
                products.append(line.strip())
        if name and products:
            full_address = ' '.join(address_lines).strip()
            address_parts = split_address(full_address)
            address_no_space_parts = [remove_spaces(part) for part in address_parts]
            extracted_data.append((
                name,
                full_address,
                address_parts[0],  # Address1
                address_parts[1],  # Address2
                address_no_space_parts[1],  # Address2 No space
                address_parts[2],  # Address3
                address_no_space_parts[2],  # Address3 No space
                address_parts[3],  # Address4
                address_no_space_parts[3],  # Address4 No space
                address_parts[4],  # Address5
                address_parts[5],  # Address6
                address_parts[6],  # Address7
                order_id,
                phone,
                ' '.join(products),
                row['file_name']
            ))
        name = None
        address_lines = []
        order_id = None
        phone = None
        products = []

    result_df = pd.DataFrame(extracted_data, columns=[
        'Name', 'Full Address', 'Address1', 'Address2', 'Address2 No space',
        'Address3', 'Address3 No space', 'Address4', 'Address4 No space', 
        'Address5', 'Address6', 'Address7', 'Order ID', 'Phone', 'Product', 'Label File Name'
    ])
    return result_df

# Function to process all PDF files from Google Drive
def process_all_pdfs_from_drive(service, file_list):
    combined_df = pd.DataFrame(columns=['pdf_extract', 'file_name'])
    for file in file_list:
        file_id = file['id']
        file_name = file['name']
        request = service.files().get_media(fileId=file_id)
        pdf_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(pdf_stream, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Reading {file_name} {int(status.progress() * 100)}% complete.")
        pdf_stream.seek(0)
        full_text = extract_text_from_pdf_stream(pdf_stream)
        print(f"Full text for {file_name}:\n{full_text[:500]}")  # Print first 500 characters for inspection

        # Improved splitting logic
        segments = full_text.strip().split("จาก")
        segments = segments[1:]  # Skip the first segment if it doesn't contain useful data
        df = pd.DataFrame(columns=['pdf_extract', 'file_name'])
        for segment in segments:
            segment = "จาก" + segment  # Add back the removed split part
            data = segment.strip().split('\n')
            new_row = pd.DataFrame({'pdf_extract': ['\n'.join(data)], 'file_name': [file_name]})
            df = pd.concat([df, new_row], ignore_index=True)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    print("Combined DataFrame:\n", combined_df.head())  # Inspect combined DataFrame
    result_df = extract_name_address_order_phone_and_products(combined_df)
    print("Result DataFrame:\n", result_df.head())  # Inspect result DataFrame
    return result_df

# Authenticate and create the service

credentials = service_account.Credentials.from_service_account_file(
    CLIENT_SECRET_FILE, scopes=SCOPES)
service = build(API_NAME, API_VERSION, credentials=credentials)
client = gspread.authorize(credentials)

# Google Drive folder ID
folder_id = '1c0vP6Y9IjhD9zjc0hy1FVXchqGVP1-AN'
query = f"'{folder_id}' in parents"

# Initialize the list of files
files = []

# Get the first page of files
response = service.files().list(q=query).execute()
files.extend(response.get('files', []))
nextPageToken = response.get('nextPageToken')

# Loop through subsequent pages of files
while nextPageToken:
    response = service.files().list(q=query, pageToken=nextPageToken).execute()
    files.extend(response.get('files', []))
    nextPageToken = response.get('nextPageToken')

# Process all PDFs from Google Drive
result_df = process_all_pdfs_from_drive(service, files)

# Add date and timestamp columns
current_date = datetime.now().date()
current_timestamp = datetime.now().isoformat()

result_df['Date'] = current_date.strftime('%Y-%m-%d')
result_df['Timestamp'] = current_timestamp

columns_order = [
    'Order ID', 'Name', 'Full Address', 'Address1', 'Address2', 'Address2 No space',
    'Address3', 'Address3 No space', 'Address4', 'Address4 No space', 'Address5',
    'Address6', 'Address7', 'Phone', 'Product', 'Label File Name', 'Date', 'Timestamp'
]
result_df = result_df[columns_order]

# Append result to Google Sheets
SHEET_URL = os.getenv("SHEET_URL")
SHEET_NAME = 'Label Extract MasterSheet'

def append_to_sheet(sheet_url, sheet_name, new_data):
    """Append data to a Google Sheet without overwriting existing data."""
    sheet = client.open_by_url(sheet_url).worksheet(sheet_name)
    
    # Read existing data
    existing_data = sheet.get_all_values()
    if existing_data:
        existing_df = pd.DataFrame(existing_data[1:], columns=existing_data[0])
    else:
        existing_df = pd.DataFrame(columns=new_data.columns)
    
    # Concatenate the existing data with the new data
    combined_df = pd.concat([existing_df, new_data], ignore_index=True)
    
    # Sort by 'Label File Name' instead of renaming
    combined_df = combined_df.sort_values(by='Label File Name', ascending=False)
    
    # Clear the existing sheet and update with the combined data
    sheet.clear()
    sheet.update([combined_df.columns.values.tolist()] + combined_df.values.tolist())

append_to_sheet(SHEET_URL, SHEET_NAME, result_df)

print(f"Data appended to {SHEET_NAME} in the Google Sheets document.")


## Merging and Updating Order Data from Google Sheets

The script effectively extracts data from Google Sheets, transforms it by merging and cleaning, and loads the transformed data back into a Google Sheet, making it a complete ETL process.

Extract Phase:

Data Retrieval: The script connects to Google Sheets using the gspread library and retrieves data from two sheets: "Update Order For Check Blacklist" (UpdateSheet) and "Label Extract MasterSheet" (LabelSheet). This is done by using the get_all_records() method, which extracts all records from the sheets.

Transform Phase:

Data Conversion: The retrieved data is converted into pandas DataFrames (df1 and df2).
Data Cleaning and Transformation:
Order ID columns are converted to strings to handle large numbers.
Specific columns from df2 are selected and renamed to avoid conflicts, creating df2_selected.
The two DataFrames are merged on the Order ID column.
Problematic values such as np.inf and -np.inf are replaced with None.
Any non-serializable float values are converted to strings.
A validation check (is_valid_dataframe) ensures there are no None values in the DataFrame.

Load Phase:

Data Backup: The existing data in UpdateSheet is backed up in case of failure.
Data Update: If the DataFrame passes validation, the script clears UpdateSheet and writes the transformed data back to the sheet, ensuring all values are converted to strings for compatibility. If validation fails, the backup data is restored.

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

# Define the scope and credentials
credentials = ServiceAccountCredentials.from_json_keyfile_name(CLIENT_SECRET_FILE, SCOPES)
client = gspread.authorize(credentials)

# Access the sheets using the URL
spreadsheet_url = os.getenv('SHEET_URL')
UpdateSheet = client.open_by_url(spreadsheet_url).worksheet('Update Order For Check Blacklist')
LabelSheet = client.open_by_url(spreadsheet_url).worksheet('Label Extract MasterSheet')

# Get the data
data1 = UpdateSheet.get_all_records()
data2 = LabelSheet.get_all_records()

# Convert to DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Convert 'Order ID' to string to handle large numbers
df1['Order ID'] = df1['Order ID'].astype(str)
df2['Order ID'] = df2['Order ID'].astype(str)

# Select specific columns from df2 and rename to avoid conflicts
df2_selected = df2[['Order ID', 'Full Address', 'Name', 'Address1', 'Address2', 'Address2 No space', 'Address3', 'Address3 No space', 'Address4', 'Address4 No space', 'Address5', 'Address6', 'Address7', 'Phone', 'Product', 'Label File Name']].rename(columns={
    'Name': 'Label Full Name',
    'Full Address': 'Label Full Address'
})

# Merge DataFrames on 'Order ID' and handle suffixes
merged_df = pd.merge(df1, df2_selected, on='Order ID', how='left', suffixes=('', '_y'))

# Handle problematic values
# Replace inf, -inf with None
merged_df.replace([np.inf, -np.inf], None, inplace=True)
# Convert any non-serializable float values to strings
merged_df = merged_df.applymap(lambda x: x if isinstance(x, (int, str)) else str(x) if isinstance(x, float) else x)

# Check if the DataFrame contains any problematic data
def is_valid_dataframe(df):
    # Check for any rows with None values (or any other criteria you want)
    return not df.isnull().values.any()

# Backup existing data in case of failure
backup_data = UpdateSheet.get_all_records()

# Only update the sheet if the DataFrame is valid
if is_valid_dataframe(merged_df):
    # Clear UpdateSheet before writing the merged DataFrame
    UpdateSheet.clear()
    
    # Convert all values to strings to ensure compatibility with Google Sheets
    formatted_data = merged_df.applymap(lambda x: str(x) if pd.notnull(x) else '')

    # Write the merged DataFrame back to UpdateSheet
    UpdateSheet.update([formatted_data.columns.values.tolist()] + formatted_data.values.tolist())
    print("Sheet updated successfully.")
else:
    print("Data validation failed. Sheet not updated.")
    # Restore backup data if needed
    backup_df = pd.DataFrame(backup_data)
    UpdateSheet.update([backup_df.columns.values.tolist()] + backup_df.values.tolist())
    print("Restoring backup data...")


## Google Sheets Data Integration and Reporting Automation


Data Processing:

Reads data from two Google Sheets: one for orders and one for blacklisted records.
Converts relevant columns to strings and checks for required columns.
Matches records from the order sheet against the blacklist based on specified conditions.
Reporting:

Filters matched records, adds date and timestamp, and cleans up any problematic values.
Updates the report sheet in Google Sheets by clearing old data and writing the new report.

In [ ]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
from datetime import datetime

# Define the URLs and sheet names
UPDATE_SHEET_URL = os.getenv('UPDATE_SHEET_URL')
UPDATE_SHEET_NAME = 'Update Order For Check Blacklist'
BLACKLIST_SHEET_URL = os.getenv('BLACKLIST_SHEET_URL')
BLACKLIST_SHEET_NAME = 'Piwpiw Bakery_ตีกลับ+Blscklist'
REPORT_SHEET_URL = os.getenv('REPORT_SHEET_URL')
REPORT_SHEET_NAME = 'Report Blacklist Update'


def read_from_sheet(sheet_url, sheet_name):
    creds = Credentials.from_service_account_file(CLIENT_SECRET_FILE, scopes=SCOPES)
    client = gspread.authorize(creds)
    sheet = client.open_by_url(sheet_url).worksheet(sheet_name)
    data = sheet.get_all_records()
    return pd.DataFrame(data)

# Function to write data to a Google Sheet
def write_to_sheet(sheet_url, sheet_name, df):
    creds = Credentials.from_service_account_file(CLIENT_SECRET_FILE, scopes=SCOPES)
    client = gspread.authorize(creds)
    sheet = client.open_by_url(sheet_url).worksheet(sheet_name)
    sheet.clear()
    sheet.update([df.columns.values.tolist()] + df.fillna('').values.tolist())

# Read data from the master sheet
update_df = read_from_sheet(UPDATE_SHEET_URL, UPDATE_SHEET_NAME)

# Read data from the blacklist sheet
blacklist_df = read_from_sheet(BLACKLIST_SHEET_URL, BLACKLIST_SHEET_NAME)

# Print column names for debugging
print("Update DataFrame Columns:", update_df.columns)
print("Blacklist DataFrame Columns:", blacklist_df.columns)

update_df['Order ID'] = update_df['Order ID'].astype(str)
blacklist_df['Order ID'] = blacklist_df['Order ID'].astype(str)

# Ensure the required columns are present
required_columns = ['Province', 'District', 'Label Full Name', 'Buyer Username']

missing_update_columns = [col for col in required_columns if col not in update_df.columns]
missing_blacklist_columns = [col for col in required_columns if col not in blacklist_df.columns]

if missing_update_columns:
    raise KeyError(f"The following columns are missing in the update DataFrame: {missing_update_columns}")
if missing_blacklist_columns:
    raise KeyError(f"The following columns are missing in the blacklist DataFrame: {missing_blacklist_columns}")

# Check for matches based on the conditions
condition_1 = update_df['Province'].isin(blacklist_df['Province']) & update_df['District'].isin(blacklist_df['District'])
condition_2 = update_df['Label Full Name'].isin(blacklist_df['Label Full Name'])
condition_3 = update_df['Buyer Username'].isin(blacklist_df['Buyer Username'])

# Combine the conditions
combined_condition = condition_1 | condition_2 | condition_3

# Filter the update_df based on the combined condition
report_blacklist = update_df[combined_condition].copy()
current_date = datetime.now().date()
current_timestamp = datetime.now().isoformat()

report_blacklist.loc[:, 'Date'] = current_date.strftime('%Y-%m-%d')
report_blacklist.loc[:, 'Timestamp'] = current_timestamp

# Check for NaN or infinite values and replace them with empty strings
report_blacklist.replace([pd.NA, float('inf'), float('-inf')], '', inplace=True)

# Write the report_blacklist DataFrame to a Google Sheet
write_to_sheet(REPORT_SHEET_URL, REPORT_SHEET_NAME, report_blacklist)
print(f"Finish Check blacklist and write report to {REPORT_SHEET_NAME}")